In [31]:
import pandas as pd
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, cross_validation
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [32]:
df = quandl.get('WIKI/GOOGL')
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [33]:
df = df[['Adj. Open','Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0


In [34]:
df.shape

(3234, 5)

In [35]:
#df['HL Percentage'] = ((df['Adj. High'] - df['Adj. Low'])/df['Adj. Low']) * 100.0
df['OC Percentage'] = ((df['Adj. Close'] - df['Adj. Open'])/df['Adj. Open']) * 100.0
df = df[['Adj. Close','Adj. High', 'Adj. Low','OC Percentage','Adj. Volume']]
df.isnull().sum()

Adj. Close       0
Adj. High        0
Adj. Low         0
OC Percentage    0
Adj. Volume      0
dtype: int64

In [36]:
df['Target'] = df['OC Percentage'].shift(-5)
df.head(10)

,Adj. Close,Adj. High,Adj. Low,OC Percentage,Adj. Volume,Target
Date,,,,,,
2004-08-19,50.322842,52.191109,48.128568,0.324968,44659000.0,2.820391
2004-08-20,54.322689,54.708881,50.405597,7.227007,22834300.0,-1.803885
2004-08-23,54.869377,56.915693,54.693835,-1.227880,18256100.0,-3.106003
2004-08-24,52.597363,55.972783,51.945350,-5.726357,15247300.0,0.048866
2004-08-25,53.164113,54.167209,52.100830,1.183658,9188600.0,-2.385589
2004-08-26,54.122070,54.142132,52.492038,2.820391,7094800.0,2.442224
2004-08-27,53.239345,54.478169,53.008633,-1.803885,6211700.0,-0.931154
2004-08-30,51.162935,52.908323,51.162935,-3.106003,5196700.0,0.564301
2004-08-31,51.343492,52.015567,51.238167,0.048866,4917800.0,1.548541


In [37]:
df.tail(6)

,Adj. Close,Adj. High,Adj. Low,OC Percentage,Adj. Volume,Target
Date,,,,,,
2017-06-15,960.18,960.6775,940.3700,1.282673,2349212.0,-0.025592
2017-06-16,958.62,959.9500,948.7000,0.074120,2484914.0,NaN
2017-06-19,975.22,979.3000,968.2000,0.574434,1449343.0,NaN
2017-06-20,968.99,980.7900,968.2000,-0.647999,1273013.0,NaN
2017-06-21,978.59,979.6700,969.1600,0.803469,1141315.0,NaN
2017-06-22,976.62,980.5000,973.3148,-0.025592,940086.0,NaN


In [38]:
Target = df['Target']
Target=Target.reset_index()
del Target['Date']
Target.head()

,Target
0,2.820391
1,-1.803885
2,-3.106003
3,0.048866
4,-2.385589


In [39]:
X = df.drop(['Target'],1)#X is DATAFRAME Without Labels
X=X.reset_index()
del X['Date']
X.head()

,Adj. Close,Adj. High,Adj. Low,OC Percentage,Adj. Volume
0,50.322842,52.191109,48.128568,0.324968,44659000.0
1,54.322689,54.708881,50.405597,7.227007,22834300.0
2,54.869377,56.915693,54.693835,-1.227880,18256100.0
3,52.597363,55.972783,51.945350,-5.726357,15247300.0
4,53.164113,54.167209,52.100830,1.183658,9188600.0


In [40]:
#Y = np.array(Target[:-5])
Y = Target[:-5]
Y.shape#Output or Labels

(3229, 1)

In [41]:
#pred = X[-5:] 
X=(X[:-5])
X.shape#Drop last Five Values

(3229, 5)

In [42]:
X.isnull().sum()
Y.isnull().sum()

Target    0
dtype: int64

In [43]:
X = preprocessing.scale(X)
Y = preprocessing.scale(Y)

In [44]:
print(type(X))
print(type(Y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [131]:
X_Train, X_Test, Y_Train, Y_Test = cross_validation.train_test_split(X, Y, test_size = 0.3)
clf = LinearRegression()
clf.fit(X_Train, Y_Train)
accu = clf.score(X_Test, Y_Test)
score = cross_val_score(clf, X_Test, Y_Test,cv=5)
print("Linear Regression "+str(accu))
print("Cross Validation Score for Linear Regression: " + str(score.mean()))

Linear Regression -0.00368938242635
Cross Validation Score for Linear Regression: 0.00189112258351
